In [ ]:
from agents import agent_loss, sampled_agent_reward, CAPM_Agent, MVP_Agent, Uniform_Agent
from trading_env.environment import TradingEnv

import dill
import random
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns 
import matplotlib.pyplot as plt

## Recall data

In [ ]:
with open('test_data.dill','rb') as dill_file:
    test_dataset = dill.load(dill_file)
test_dates = test_dataset['dates']
test_data = test_dataset['data']

n_stocks = test_data.shape[0]

env = TradingEnv(test_data)
start = env._start_tick

## Recall trained agent

In [ ]:
with open('dpm_agent.dill','rb') as dill_file:
    dpm_agent = dill.load(dill_file)

capm_agent = CAPM_Agent()
mvp_agent = MVP_Agent()
uniform_agent = Uniform_Agent()

### Run a single simulation on all stock data

In [ ]:
env = TradingEnv(test_data)

dpm_loss = agent_loss(env,dpm_agent)
dpm_val_hist = env.portfolio_value_hist

capm_loss = agent_loss(env,capm_agent)
capm_val_hist = env.portfolio_value_hist

mvp_loss = agent_loss(env,mvp_agent)
mvp_val_hist = env.portfolio_value_hist

uniform_loss = agent_loss(env,uniform_agent)
uniform_val_hist = env.portfolio_value_hist


In [ ]:
plt.plot(test_dates[start:],dpm_val_hist,label='Deep Portfolio Management')
plt.plot(test_dates[start:],capm_val_hist,label='Capital Asset Pricing Model')
plt.plot(test_dates[start:],mvp_val_hist,label='Minimum Variance Portfolio')
plt.plot(test_dates[start:],uniform_val_hist,label='Uniform Portfolio Weights')
plt.xticks(rotation=45)
plt.legend(loc='upper right',bbox_to_anchor=(1.6, 1) )
plt.title('Comparison of several investment strategies.')
plt.show()

## Statistical analysis

One simulation is interesting to view but it is important to compare the statistical performance of the agents. Here we will simulate several trials and perform a simple t-test on the results to compare agents. Note the randomization is in the subset of stocks chosen in each trial. On each trial we select with replacement a configuration of 10 out of 36 stocks on which to run simulations. There is a total of ${36 \choose 10} \approx 9.1 \times 10^{9}$ possible configurations. On each trial the configuration is fed into each agent, thus yielding a paired t-test experiment design.

In [ ]:
TRIALS = 100
n_group_size = 8

capm_agent = CAPM_Agent()
mvp_agent = MVP_Agent()
uniform_agent = Uniform_Agent()
agents = [dpm_agent,
            capm_agent,
            mvp_agent,
            uniform_agent]

rewards = sampled_agent_reward(TradingEnv,test_data,agents,n_group_size,TRIALS)     

rewards_df = pd.DataFrame(rewards).astype(float)

In [ ]:
print(f'The rewards_df dataframe contains {rewards_df.shape[0]} rows ',end='') 
print('representing normalized annual returns for each agent for each ', end = '')
print('selected configuration.' )
rewards_df.head()

## Visualize the results

In [ ]:
#final_vals_df = pd.DataFrame(final_vals).astype(float)

sns.boxplot(data=rewards_df)
plt.title('Average annual returns.')
plt.show()


In [ ]:
print('Mean annual returns for each agent.')
rewards_df.mean()

In [ ]:
rewards_df.hist()
plt.show()

## t-test

It seems from the above visualizations that the DeepPortfolio agent outperforms the Uniform agent, but can we be sure about that?

We perform a t-tests with the alternative hypotheses that DeepPortfolio outperforms the Uniform portfolio.

In [ ]:
stats.ttest_rel(rewards_df['DPM_Agent'],rewards_df['Uniform_Agent'] ,alternative = 'greater')

The p value is very small, so we can easily reject the null hypothesis and conclude DeepPortfolio outperforms the Uniform agent.
 
